# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidYellowLeft.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines):
    color = [255, 0, 0]
    thickness = 2
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    # line_img = np.zeros(img.shape, dtype=np.uint8)
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
    return img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns hough lines.
    """
    return cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
image_files = os.listdir("test_images/")

image_number = 1
image = mpimg.imread('/'.join(['test_images', image_files[image_number]]))

def find_raw_lines(image):
    """
    Returns numpy array of lines [(x1, y1, x2, y2)]
    """
    img = grayscale(image)
    
    img = gaussian_blur(img, 5)
    img = canny(img, 50, 150)

    width = image.shape[1]
    height = image.shape[0]
    clip_top = 0.6
    clip_left_top = 0.46
    clip_left_bottom = 0.08

    img = region_of_interest(img, np.int32(np.array([[
            [width * clip_left_top, height * clip_top],
            [width * (1 - clip_left_top), height * clip_top],
            [width * (1 - clip_left_bottom), height],
            [width * clip_left_bottom, height]
        ]], dtype='int32')))

    rho = 2
    degrees = 2
    theta = degrees * np.pi / 180
    threshold = 5
    min_line = 20
    max_gap = 10

    return hough_lines(img, rho, theta, threshold, min_line, max_gap);

def get_slope_and_intercepts(line, image_height):
    """
    Takes line as (x1, y1, x2, y2) from numpy array
    Returns slope, y intercept, baseline intercept x value
    """
    x1, y1, x2, y2 = line[0]
    slope = (y2 - y1) / (x2 - x1)
    y_intercept = y1 - (slope * x1)
    baseline_intercept_x = (image_height - y_intercept) / slope
    return slope, y_intercept, baseline_intercept_x 

def slope_buckets(lines, min_slope, max_slope, threshold, image_height):
    """
    Takes lines as numpy array of (x1, y1, x2, y2)
    threshold = deviation of bucket from average
    min_slope -> discard lines below
    max_slope -> discard lines above
    max_slope must be > min_slope
    Returns [[[slope, y_intercept, baseline_x_intercept, line]]]
    a little akward to pass all this data through, but keeps from having
    to recalculate it
    """
    # this is O(n^2) right now: use bisect or other custom binary search later
    buckets = []
    for line in lines:
        slope, y_intercept, baseline_intercept_x = get_slope_and_intercepts(line, image_height)
        if slope < min_slope or slope > max_slope:
            continue
        line_entry = [slope, y_intercept, baseline_intercept_x, line]
        if not buckets:
            buckets.append([slope, slope, 1, [line_entry]])
        bucket_found = False
        for bucket in buckets:
            if abs(slope - bucket[0]) <= threshold:
                bucket[3].append(line_entry)
                bucket[2] += 1 # count
                bucket[1] += slope # sum
                bucket[0] = bucket[1] / bucket[2] # average
                bucket_found = True
                break;
        if not bucket_found:
            buckets.append([slope, slope, 1, [line_entry]])
    return [bucket[3] for bucket in buckets]

def intersects_baseline(line, image_width):
    """
    Takes [slope, y_intercept, baseline_intercept_x, line]
    Returns True if line intercepts baseline
    """
    x = line[2]
    return x >= 0 and x <= image_width

def filter_buckets_by_baseline_intercept(buckets, image_width):
    """
    Takes buckets from slope_buckets
    Returns buckets filtered by whether lines would intersect baseline of image
    """
    return [[line for line in bucket if intersects_baseline(line, image_width)] for bucket in buckets]
    
def consolidate_lines(lines, image_height):
    """
    Takes array of lines with metadata [slope, y_intercept, baseline_x_intercept, line]
    Returns a line averaging these
    Simple formula: take baseline intercept of line with max y point
    and point with min y
    """
    first = lines[0][3][0]
    x1, y1, x2, y2 = first
    max_y = image_height
    # only consider lines near base for base intercept
    intercept_y_threshold = 0.9
    intercept_lines = []
    while(not intercept_lines):
        y_threshold = image_height * intercept_y_threshold
        intercept_lines = [line[2] for line in lines 
                           if line[3][0][1] > y_threshold or
                           line[3][0][3] > y_threshold]
        intercept_y_threshold -= 0.1
    
    max_x = sum(intercept_lines) / len(intercept_lines)
    min_y = min(y1, y2)
    min_x = x1 if y1 == min_y else x2
    for line in lines:
        for x1, y1, x2, y2 in line[3]:
            min_y_local = min(y1, y2)
            if min_y_local < min_y:
                min_y = min_y_local
                min_x = x1 if y1 == min_y_local else x2
    line = [min_x, min_y, max_x, max_y] if min_x < max_x else [max_x, max_y, min_x, min_y]
    return np.array([line], dtype='int32')   
    
width = image.shape[1]
height = image.shape[0]
lines = find_raw_lines(image);
buckets = slope_buckets(lines, -2.0, 2.0, 0.2, height)
buckets = filter_buckets_by_baseline_intercept(buckets, width)
# need this filter to get rid of stray lines
buckets = [bucket for bucket in buckets if len(bucket) >= 3]
derived_lines = [consolidate_lines(bucket, height) for bucket in buckets]
    
img = draw_lines(image, derived_lines)

plt.imshow(img)

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    def find_raw_lines2(image):
        """
        Returns numpy array of lines [(x1, y1, x2, y2)]
        """
        # take the max color channel
        r, g, b = cv2.split(image)
        img = cv2.max(r, g)

        img = gaussian_blur(img, 5)
        img = canny(img, 50, 150)

        width = image.shape[1]
        height = image.shape[0]
        clip_top = 0.6
        clip_left_top = 0.46
        clip_left_bottom = 0.08
    
        img = region_of_interest(img, np.int32(np.array([[
                [width * clip_left_top, height * clip_top],
                [width * (1 - clip_left_top), height * clip_top],
                [width * (1 - clip_left_bottom), height],
                [width * clip_left_bottom, height]
            ]], dtype='int32')))

        rho = 2
        degrees = 2
        theta = degrees * np.pi / 180
        threshold = 5
        min_line = 15
        max_gap = 10

        return hough_lines(img, rho, theta, threshold, min_line, max_gap);

    def get_slope_and_intercepts(line, image_height):
        """
        Takes line as (x1, y1, x2, y2) from numpy array
        Returns slope, y intercept, baseline intercept x value
        """
        x1, y1, x2, y2 = line[0]
        slope = (y2 - y1) / (x2 - x1)
        y_intercept = y1 - (slope * x1)
        baseline_intercept_x = (image_height - y_intercept) / slope
        return slope, y_intercept, baseline_intercept_x 

    def slope_buckets2(lines, max_slope, min_slope_from_0, threshold, image_width, image_height):
        """
        Takes lines as numpy array of (x1, y1, x2, y2)
        threshold = deviation of bucket from average
        min_slope -> discard lines below
        max_slope -> discard lines above
        min_slope_from_0 -> discard line with slope within this value from 0
        max_slope must be > min_slope
        Returns [[[slope, y_intercept, baseline_x_intercept, line]]]
        a little akward to pass all this data through, but keeps from having
        to recalculate it
        """
        # this is O(n^2) right now: use bisect or other custom binary search later
        buckets = []
        mid_x = width / 2.0
        for line in lines:
            slope, y_intercept, baseline_intercept_x = get_slope_and_intercepts(line, image_height)
            # remove lines crossing the midline
            x1, y1, x2, y2 = line[0]
            if (x1 < mid_x and x2 > mid_x) or (x2 < mid_x and x1 > mid_x):
                continue
            abs_slope = abs(slope)
            if abs_slope < min_slope_from_0 or abs_slope > max_slope:
                continue
            line_entry = [slope, y_intercept, baseline_intercept_x, line]
            if not buckets:
                buckets.append([slope, slope, 1, [line_entry]])
            bucket_found = False
            for bucket in buckets:
                if abs(slope - bucket[0]) <= threshold:
                    bucket[3].append(line_entry)
                    bucket[2] += 1 # count
                    bucket[1] += slope # sum
                    bucket[0] = bucket[1] / bucket[2] # average
                    bucket_found = True
                    break;
            if not bucket_found:
                buckets.append([slope, slope, 1, [line_entry]])
        return [bucket[3] for bucket in buckets]

    def intersects_baseline(line, image_width):
        """
        Takes [slope, y_intercept, baseline_intercept_x, line]
        Returns True if line intercepts baseline
        """
        x = line[2]
        return x >= 0 and x <= image_width
    
    def intersects_center_region(line, image_width, center_fraction):
        x = line[2]
        center_width = image_width * (1 - center_fraction) / 2
        center_min_x = center_width
        center_max_x = image_width - center_width
        return x >= center_min_x and x <= center_max_x
    
    def filter_buckets_by_baseline_intercept2(buckets, image_width):
        """
        Takes buckets from slope_buckets
        Returns buckets filtered by whether lines would intersect baseline of image
        """
        return [[line for line in bucket if intersects_baseline(line, image_width) and not intersects_center_region(line, image_width, 0.4)] for bucket in buckets]
    
    def consolidate_lines2(lines, image_height):
        """
        Takes array of lines with metadata [slope, y_intercept, baseline_x_intercept, line]
        Returns a line averaging these
        Simple formula: take baseline intercept of line with max y point
        and point with min y
        """
        first = lines[0][3][0]
        x1, y1, x2, y2 = first
        max_y = image_height
        # only consider lines near base for base intercept
        intercept_y_threshold = 0.9
        intercept_lines = []
        while(not intercept_lines):
            y_threshold = image_height * intercept_y_threshold
            intercept_lines = [line[2] for line in lines 
                               if line[3][0][1] > y_threshold or
                               line[3][0][3] > y_threshold]
            intercept_y_threshold -= 0.1
    
        max_x = sum(intercept_lines) / len(intercept_lines)
        min_y = min(y1, y2)
        min_x = x1 if y1 == min_y else x2
        for line in lines:
            for x1, y1, x2, y2 in line[3]:
                min_y_local = min(y1, y2)
                if min_y_local < min_y:
                    min_y = min_y_local
                    min_x = x1 if y1 == min_y_local else x2
        line = [min_x, min_y, max_x, max_y] if min_x < max_x else [max_x, max_y, min_x, min_y]
        return np.array([line], dtype='int32')
    
    def get_2(lines):
        """
        Takes array of lines
        Returns array of lines, the 2 most horizontal
        """
        if (len(lines) < 3):
            return lines
        slopes = [[(y2 - y1) / (x2 - x1), line] for line in lines for x1, y1, x2, y2 in line]
        negative_slope_lines = [[abs(line[0]), line[1]] for line in slopes if line[0] < 0]
        positive_slope_lines = [[abs(line[0]), line[1]] for line in slopes if line[0] > 0]
        
        def get_min_slope_line(lines):
            if (not lines): return []
            if (len(lines) == 1): return lines[0][1]

            min_slope = lines[0][0]
            min_slope_line = lines[0][1]
            for i in range(len(lines)):
                line = lines[i]
                slope = line[0]
                if slope < min_slope:
                    min_slope = slope
                    min_slope_line = line[1]
            return min_slope_line
        
        lines = []
        min_line = get_min_slope_line(negative_slope_lines)
        max_line = get_min_slope_line(positive_slope_lines)
        if len(min_line): lines.append(min_line)
        if len(max_line): lines.append(max_line)
        return lines
    
    
    width = image.shape[1]
    height = image.shape[0]
    
    # method 1 - for straight line videos
    # uses functions from image test section
    lines = find_raw_lines(image);
    buckets = slope_buckets(lines, -2.0, 2.0, 0.2, height)
    buckets = filter_buckets_by_baseline_intercept(buckets, width)
    buckets = [bucket for bucket in buckets if len(bucket) >= 3]
    derived_lines = [consolidate_lines(bucket, height) for bucket in buckets]
    
    result = draw_lines(image, derived_lines)
    
    # method 2 - for challenge video
    # uses functions from this block
    #lines = find_raw_lines2(image);
    #buckets = slope_buckets2(lines, 1.5, 0.7, 0.1, width, height)
    #buckets = filter_buckets_by_baseline_intercept2(buckets, width)
    #buckets = [bucket for bucket in buckets if len(bucket) > 0]
    #derived_lines = [consolidate_lines2(bucket, height) for bucket in buckets]
    
    #result = draw_lines(image, get_2(derived_lines))

    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

 ###Reflections
 
 Light patches and contrast differences in the road made lane detection more difficult. I tried picking the max color channel, and while this allowed picking up some lines one the light patch I couldn't previously, it also picked up unwanted noise from the road.
 
 It seems like the detection is very dependent on the aspect ratio of the camera and region of interest. Would be nice to get the method independent of this.
 
 For curving lanes, simple combination of lines didn't quite produce the desired result.  I would like to try something more like finding connected components in a graph, starting from lines close to the base line.
 
 What I learned most is that tuning to real conditions is hard. Even tuning frame by frame would be no guarantee of picking up lanes under different conditions. And abnormal conditions would throw these methods off quite easily. I imagine a large number of training conditions would be needed to make this detection work smoothly.
